In [8]:
import pandas as pd
import numpy as np
from fastapi import FastAPI
import re

app = FastAPI()

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [3]:
pd.set_option("max_colwidth", 270)
#parquet_file = 'https://raw.githubusercontent.com/Dforko/Moviex/main/dbOK_small.snappy.parquet'
db = pd.read_parquet('dbOK_small.snappy.parquet', engine='auto')


In [5]:
db.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 44285 entries, 0 to 45458
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   index                 44285 non-null  int64         
 1   id                    44285 non-null  int64         
 2   title                 44285 non-null  string        
 3   genres                44285 non-null  string        
 4   overview              44285 non-null  string        
 5   tagline               44285 non-null  string        
 6   spoken_languages      44285 non-null  string        
 7   original_language     44285 non-null  string        
 8   budget                44285 non-null  int64         
 9   revenue               44285 non-null  int64         
 10  popularity            44285 non-null  float64       
 11  release_date          44285 non-null  datetime64[ns]
 12  runtime               44285 non-null  int64         
 13  production_compa

In [14]:

db['title']=db['title'].astype('string')
db[ 'genres']=db[ 'genres'].astype('string')
db['overview']=db['overview'].astype('string')
db['tagline']=db[ 'tagline'].astype('string')
db['spoken_languages']=db[ 'spoken_languages'].astype('string')
db['original_language']=db['original_language'].astype('string')
db['production_companies']=db[ 'production_companies'].astype('string')
db['production_countries']=db[ 'production_countries'].astype('string')
db['cast']=db[ 'cast'].astype('string')
db['director']=db['director'].astype('string')
db['crew1']=db[ 'crew1'].astype('string')
db['crew2']=db[ 'crew2'].astype('string')
db['crew3']=db[ 'crew3'].astype('string')
db['crew4']=db[ 'crew4'].astype('string')
db['release_date']=pd.to_datetime(db['release_date'],errors='coerce')

In [15]:
db['genres']=db['genres'].fillna('*Unavailable')
db['overview']=db['overview'].fillna('*Unavailable')
db['tagline']=db['tagline'].fillna('*Unavailable')
db['spoken_languages']=db['spoken_languages'].fillna('*Unavailable')
db['production_companies']=db['production_companies'].fillna('*Unavailable')
db['production_countries']=db['production_countries'].fillna('*Unavailable')

db['cast']=db['cast'].fillna('*Unavailable')
db['crew1']=db['crew1'].fillna('*Unavailable')
db['crew2']=db['crew2'].fillna('*Unavailable')
db['crew3']=db['crew3'].fillna('*Unavailable')
db['crew4']=db['crew4'].fillna('*Unavailable')
db['cast'] = db['cast'].str.replace(r'[^\x20-\x7E]', '')


C:\Users\dforc\AppData\Local\Temp\ipykernel_17116\1041640844.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  db['cast'] = db['cast'].str.replace(r'[^\x20-\x7E]', '')


In [16]:
db['director']=db['director'].fillna('*Unavailable')
db['cast'] = db['cast'].str.replace(r'[^\x20-\x7E]', '')
db['cast'] = db['cast'].str.lstrip()

C:\Users\dforc\AppData\Local\Temp\ipykernel_17116\51532510.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  db['cast'] = db['cast'].str.replace(r'[^\x20-\x7E]', '')


In [18]:
def cantidad_filmaciones(Mes:str):
    m_dic = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
        'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    nro_mes = m_dic.get(Mes)
    if nro_mes is None:
        return(f"{Mes} no es un mes válido.")
        return
    return {'En': Mes, 'Fueron estrenadas la cantidad de':sum(1 for date in db['release_date'] if date.month == nro_mes)}

In [20]:
cantidad_filmaciones('May')

'May no es un mes válido.'

In [8]:
db.loc[(db['budget'] > 0) & (db['budget'] < 100), 'budget'] *= 100000

In [9]:
db=db.rename(columns={"Unnamed: 0": "index"})


In [ ]:
db.drop('Unnamed: 0.1', axis=1, inplace=True)

In [ ]:
db.info()

In [ ]:
db.to_parquet('dbOK.parquet')

In [ ]:
db.head()

In [ ]:
#OPTIMIZADA

app = FastAPI()

@app.get('/')
def cantidad_filmaciones_mes(Mes):
    m_dic = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,'Julio': 7,
             'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}
    try:
        nro_mes = m_dic[Mes]
    except KeyError:
        print(f"{Mes} no es un mes válido.")
        return
    return (db['release_date'].dt.month == m_dic[Mes]).sum()

In [ ]:
#OPTIMIZADA 2
def cantidad_filmaciones(Mes):
    m_dic = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
        'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    nro_mes = m_dic.get(Mes)
    if nro_mes is None:
        print(f"{Mes} no es un mes válido.")
        return
    return sum(1 for date in db['release_date'] if date.month == nro_mes)


In [ ]:
cantidad_filmaciones('Mayo')

3345

In [ ]:
#OPTIMIZADA para List 



@app.get('/')
def cantidad_filmaciones_mes(meses):
    m_dic = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
        'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    counts = 0
    for mes in meses:
        try:
            nro_mes = m_dic[mes]
        except KeyError:
            print(f"{mes} no es un mes válido.")
            counts.append(None)
            continue
        count = (db['release_date'].dt.month == nro_mes).sum()
        counts+=count 
    return counts



In [ ]:
cantidad_filmaciones_mes(Mes)

45459

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [36]:


@app.get('/cantidad_filmaciones_dia/{Dia}')
def cantidad_filmaciones_dia(Dia:str):

    # Creamos diccionario para normalizar
    days = {
    'Lunes': 'Monday',
    'Martes': 'Tuesday',
    'Miércoles': 'Wednesday',
    'Jueves': 'Thursday',
    'Viernes': 'Friday',
    'Sábado': 'Saturday',
    'Domingo': 'Sunday'}
    day = days[Dia]
    lista_peliculas_day = db[db['release_date'].dt.day_name() == day]
    respuesta = lista_peliculas_day.shape[0]
    return {
        'La cantidad de': respuesta,
        'películas fueron estrenadas en los días': Dia    }

In [38]:
cantidad_filmaciones_dia("Lues")

KeyError: 'Lues'

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

In [13]:


@app.get('/')
def  score_titulo(titulo_de_la_filmación):    
    # Find the row that matches the movie title
    movie_info = db.loc[db['title'] == titulo_de_la_filmación]
    if  titulo_de_la_filmación not in movie_info.values:
        print(f"{titulo_de_la_filmación} no es un título válido. Chequea que el titulo_de_la_filmación sea el correcto, la ortografía y mayúsculas.")
        return  
    return f'La película {movie_info.title.values[0]} fue estrenada en el año { movie_info.release_year.values[0]} con un score/popularidad de {movie_info.popularity.values[0]}'

In [14]:
score_titulo('Atanarjuat: The Fast Runner')

'La película Atanarjuat: The Fast Runner fue estrenada en el año 2002 con un score/popularidad de 1.135061'

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [15]:
app = FastAPI()

@app.get('/')

def votos_titulo( titulo_de_la_filmación ):
    # Find the row that matches the movie title
    movie_info = db.loc[db['title'] == titulo_de_la_filmación]
    if  titulo_de_la_filmación not in movie_info.values:
        print(f"{titulo_de_la_filmación} no es un título válido. Chequea que el título sea el correcto, la ortografía y mayúsculas.")
        return
    if  movie_info.vote_count.values[0] < 2000:
       return f"{titulo_de_la_filmación} no posee suficientes valoraciones."
    else:
        return f'La película {movie_info.title.values[0]} fue estrenada en el año { movie_info.release_year.values[0]}. La misma cuenta con un total de { movie_info.vote_count.values[0]} valoraciones, con un promedio de { movie_info.vote_average.values[0]}'  
    

In [16]:
votos_titulo('The Dark Knight')

'La película The Dark Knight fue estrenada en el año 2008. La misma cuenta con un total de 12269 valoraciones, con un promedio de 8.3'

In [1]:
user_input = input("Digite película: ")
def votos_titulo( user_input ):
    teamnames= db['title'].values
    movie_input  = filter(lambda x: x.startswith(user_input), teamnames)
    if len(movie_input ) > 1:
    # Deal with more that one team.
        print('There are more than one team starting with "{0}"'.format(user_input))
        print('Select the team from choices: ')
    for index, name in enumerate(movie_input ):
        print("{0}: {1}".format(index, name))
        index = input("Enter choice number: ")
    # You might want to handle IndexError exception here.
        print('Selected team: {0}'.format(movie_input [index]))
    else:
    # Only one team found, so print that team.
        print (movie_input [0])



In [ ]:
votos_titulo( user_input )

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [372]:
app = FastAPI()

@app.get('/')
def get_actor( nombre_actor ):
    actor_info = db.loc[db['cast'].str.contains(nombre_actor)]
    actor_cant_movies=actor_info['cast'].count()
    actor_return =round(actor_info['return'].sum(),2)
    if  nombre_actor not in actor_info.values:
        print(f"{nombre_actor} no es un nombre válido. Chequea la ortografía y mayúsculas.")
        return
    return f'El actor/la actriz {nombre_actor} ha participado de {actor_cant_movies} filmaciones, conseguiendo un retorno de {actor_return} con un promedio de {round(actor_return/actor_cant_movies,2)} por filmación'

In [ ]:
actor_info = db.loc[db['cast'].str.contains('Steve McQueen')]
actor_cant_movies=actor_info['cast'].count
actor_return =actor_info['return'].sum()
print(actor_cant_movies,actor_return)


In [324]:
get_actor('Robert Downey Jr.')

'El actor/la actriz Robert Downey Jr. ha participado de 70 filmaciones, conseguiendo un retorno de 12396458.14 con un promedio de 177092.26 por filmación'

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [ ]:
db.tail(12)

In [380]:
app = FastAPI()

@app.get('/')

def get_director(nombre_director):
    # Filtrar las filas que corresponden al director especificado
    director_db = db[db['director'] == nombre_director]
    
    if nombre_director not in db['director'].values:
        print(f"{nombre_director} no es un nombre válido. Chequea la ortografía y mayúsculas.")
        return
    
    # Calcular la suma de la columna 'Retorno'
    suma_retorno = director_db['return'].sum()
    
    # Crear un DataFrame con las columnas especificadas
    resultado = director_db[['title', 'release_year', 'return', 'budget', 'revenue']]
    
    
    # Renombrar las columnas del DataFrame
    resultado = resultado.rename(columns={'title': 'Título', 'release_year': 'Año', 'return': 'Retorno', 'budget': 'Costo', 'revenue': 'Ganancia'})
    # Ordenar el DataFrame por la columna 'Año' en orden ascendente
    resultado = resultado.sort_values(by='Año', ascending=True)
    # Devolver la suma de retorno y el DataFrame resultado
    return f'El retorno total de {nombre_director} es de {round(suma_retorno, 2)}', 'Estas son sus películas, año de lanzamiento, retorno individual, costo y ganancia:', resultado


In [ ]:
get_director('Roger Corman')

In [ ]:
db = db.rename(columns={'title':'Título', 'release_year':'Año', "return":'Retorno' , 'budget':'Costo', 'revenue': 'Ganancia'})

## Content Based Recommender

def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

I will build two Content Based Recommenders based on:

Movie Overviews and Taglines
Movie Cast, Crew, Keywords and Genre
Also, as mentioned in the introduction, I will be using a subset of all the movies available to us due to limiting computing power available to me.

In [3]:
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:

# loading the data from the csv file to apandas dataframe
movies_data = db

# selecting the relevant features for recommendation

selected_features = ['title','genres','overview','tagline','director']

# combining all the 5 selected features

combined_features = movies_data['title']+''+movies_data['genres']+' '+movies_data['overview']+' '+movies_data['tagline']+' '+movies_data['director']

# converting the text data to feature vectors

vectorizer = TfidfVectorizer(ngram_range=(1,2))

feature_vectors = vectorizer.fit_transform(combined_features)



## Cosine Similarity

In [5]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [6]:

np.save('similarity.npy', similarity)

In [8]:
saved_array = np.load('similarity.npy')
print(saved_array)

[[1.         0.01031757 0.01163132 ... 0.00266651 0.01175589 0.00548818]
 [0.01031757 1.         0.01555041 ... 0.01649255 0.02454345 0.00948163]
 [0.01163132 0.01555041 1.         ... 0.00244511 0.01595727 0.00576674]
 ...
 [0.00266651 0.01649255 0.00244511 ... 1.         0.00641488 0.00409453]
 [0.01175589 0.02454345 0.01595727 ... 0.00641488 1.         0.01842062]
 [0.00548818 0.00948163 0.00576674 ... 0.00409453 0.01842062 1.        ]]


In [9]:
print(saved_array.shape)

(45456, 45456)


In [29]:
db.columns


Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'title', 'genres', 'overview',
       'tagline', 'spoken_languages', 'original_language', 'budget', 'revenue',
       'popularity', 'release_date', 'runtime', 'production_companies',
       'production_countries', 'vote_average', 'vote_count', 'release_year',
       'return', 'cast', 'director', 'crew1', 'crew2', 'crew3', 'crew4',
       'titulo'],
      dtype='object')

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = db['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = db[db.title == close_match]['index'].values[0]

similarity_score = list(enumerate(saved_array[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = db[db.index==index]['title'].values[0]
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

In [12]:
recomendacion("batman")

Movies suggested for you : 



IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
@app.get('/cantidad_filmaciones_mes/{mes}')
def cantidad_filmaciones_mes(mes:str):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
    return {'mes':mes, 'cantidad':respuesta}

@app.get('/cantidad_filmaciones_dia{dia}')
def cantidad_filmaciones_dia(dia:str):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente'''
    return {'dia':dia, 'cantidad':respuesta}

@app.get('/score_titulo/{titulo}')
def score_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score'''
    return {'titulo':titulo, 'anio':respuesta, 'popularidad':respuesta}



@app.get('/get_director/{nombre_director}')
def get_director(nombre_director:str):
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''
    return {'director':nombre_director, 'retorno_total_director':respuesta, 
    'peliculas':respuesta, 'anio':respuesta,, 'retorno_pelicula':respuesta, 
    'budget_pelicula':respuesta, 'revenue_pelicula':respuesta}

# ML
@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    return {'lista recomendada': respuesta}

@app.get('/votos_titulo/{titulo}')
def votos_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
    La misma variable deberá de contar con al menos 2000 valoraciones, 
    caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'''
    return {'titulo':titulo, 'anio':respuesta, 'voto_total':respuesta, 'voto_promedio':respuesta}

@app.get('/get_actor/{nombre_actor}')
def get_actor(nombre_actor:str):
    '''Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, la cantidad de películas que en las que ha participado y el promedio de retorno'''
    return {'actor':nombre_actor, 'cantidad_filmaciones':respuesta, 'retorno_total':respuesta, 'retorno_promedio':respuesta}

In [ ]:
# function that takes in the character name as input and returns the top 10 recommended characters
def recommendations(name, cosine_sim = cosine_sim):
    
    recommended_names = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most characters
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the names of the best 10 matching characters
    for i in top_10_indexes:
        recommended_names.append(list(df.index)[i])
        
    return recommended_names

# working results which for dataset are pretty good 

recommendations('Blues Traveler')

['G-Love & The Special Sauce',
 'Phish',
 'Spin Doctors',
 'Grace Potter and the Nocturnals',
 'Jason Mraz',
 'Pearl Jam',
 'Dave Matthews Band',
 'Lukas Nelson & Promise of the Real ',
 'Vonda Shepard',
 'Goo Goo Dolls']